# PyDipAPI - Async API Tutorial

Dieses Notebook demonstriert die asynchronen Features von PyDipAPI für bessere Performance.

## Was Sie lernen werden:
1. **AsyncDipAnfrage** - Asynchroner Client
2. **Context Managers** - Ressourcen-Management
3. **Performance-Vergleich** - Sync vs. Async
4. **Parallele Anfragen** - Mehrere Requests gleichzeitig
5. **Error Handling** - Async-spezifische Fehlerbehandlung

In [ ]:
import sys
import os
import asyncio
import time
sys.path.insert(0, os.path.abspath('..'))

from pydipapi import DipAnfrage
from pydipapi.async_api import AsyncDipAnfrage

# API-Konfiguration
API_KEY = "HIER_IHREN_API_SCHLUESSEL_EINFUEGEN"
sync_client = DipAnfrage(api_key=API_KEY)

print("✅ Async API Tutorial bereit!")

## AsyncDipAnfrage - Grundlagen

In [ ]:
# Async Client testen
async def test_async_basic():
    print("🚀 Teste AsyncDipAnfrage...")
    
    async with AsyncDipAnfrage(api_key=API_KEY) as async_client:
        try:
            persons = await async_client.get_person(anzahl=3)
            print(f"✅ {len(persons)} Personen asynchron geladen")
            
            if persons:
                person = persons[0]
                name = f"{person.get('vorname', '')} {person.get('nachname', '')}"
                print(f"Erste Person: {name.strip()}")
                
        except Exception as e:
            print(f"❌ Fehler: {e}")
    
    print("✅ Async Client ordnungsgemäß geschlossen")

# Async Funktion ausführen
await test_async_basic()

## Performance-Vergleich: Sync vs. Async

In [ ]:
# Performance-Vergleich
async def performance_test():
    num_requests = 4
    print(f"⚡ Performance-Test mit {num_requests} Anfragen")
    print("=" * 50)
    
    # Synchrone Anfragen (sequenziell)
    print(f"📊 Synchrone Anfragen...")
    start_time = time.time()
    
    sync_results = []
    for i in range(num_requests):
        try:
            result = sync_client.get_person(anzahl=2)
            sync_results.append(len(result) if result else 0)
        except Exception as e:
            print(f"Sync Fehler {i+1}: {e}")
            sync_results.append(0)
    
    sync_time = time.time() - start_time
    print(f"✅ Sync Zeit: {sync_time:.2f} Sekunden")
    
    # Asynchrone Anfragen (parallel)
    print(f"\n🚀 Asynchrone Anfragen...")
    start_time = time.time()
    
    async def fetch_async(client, request_id):
        try:
            result = await client.get_person(anzahl=2)
            return len(result) if result else 0
        except Exception as e:
            print(f"Async Fehler {request_id}: {e}")
            return 0
    
    async with AsyncDipAnfrage(api_key=API_KEY) as async_client:
        tasks = [fetch_async(async_client, i+1) for i in range(num_requests)]
        async_results = await asyncio.gather(*tasks)
    
    async_time = time.time() - start_time
    print(f"✅ Async Zeit: {async_time:.2f} Sekunden")
    
    # Ergebnisse
    print(f"\n📈 Performance-Gewinn:")
    if async_time > 0:
        speedup = sync_time / async_time
        print(f"- Async ist {speedup:.1f}x schneller!")
        print(f"- Zeitersparnis: {sync_time - async_time:.2f} Sekunden")
    
    print(f"\n📊 Ergebnisse:")
    print(f"- Sync: {sync_results}")
    print(f"- Async: {async_results}")

await performance_test()

## Erweiterte Async-Patterns

In [ ]:
# Verschiedene Datentypen parallel abrufen
async def mixed_data_fetching():
    print("🎯 Verschiedene Datentypen parallel abrufen...")
    
    async with AsyncDipAnfrage(api_key=API_KEY) as client:
        # Alle Anfragen parallel starten
        tasks = [
            client.get_person(anzahl=3),
            client.get_drucksache(anzahl=3),
            client.get_aktivitaet(anzahl=3)
        ]
        
        try:
            persons, documents, activities = await asyncio.gather(*tasks)
            
            print(f"✅ Parallel geladen:")
            print(f"  - Personen: {len(persons) if persons else 0}")
            print(f"  - Dokumente: {len(documents) if documents else 0}")
            print(f"  - Aktivitäten: {len(activities) if activities else 0}")
            
        except Exception as e:
            print(f"❌ Fehler beim parallelen Laden: {e}")

await mixed_data_fetching()

## Error Handling in Async Context

In [ ]:
# Robuste Fehlerbehandlung
async def robust_async_fetching():
    print("🛡️ Teste robuste Fehlerbehandlung...")
    
    async def safe_fetch(client, fetch_func, name):
        try:
            result = await fetch_func()
            print(f"✅ {name}: {len(result) if result else 0} Elemente")
            return result
        except Exception as e:
            print(f"❌ {name} Fehler: {e}")
            return []
    
    async with AsyncDipAnfrage(api_key=API_KEY) as client:
        # Verschiedene Anfragen mit individueller Fehlerbehandlung
        tasks = [
            safe_fetch(client, lambda: client.get_person(anzahl=2), "Personen"),
            safe_fetch(client, lambda: client.get_drucksache(anzahl=2), "Dokumente"),
            safe_fetch(client, lambda: client.get_aktivitaet(anzahl=2), "Aktivitäten")
        ]
        
        results = await asyncio.gather(*tasks, return_exceptions=True)
        
        successful_requests = sum(1 for r in results if not isinstance(r, Exception))
        print(f"\n📊 Erfolgreiche Anfragen: {successful_requests}/{len(tasks)}")

await robust_async_fetching()

## Zusammenfassung

✅ **Async API erfolgreich getestet!**

### Gelernte Konzepte:
- **AsyncDipAnfrage**: Asynchroner Client für parallele Anfragen
- **Context Manager**: Automatisches Ressourcen-Management
- **Performance**: Erhebliche Geschwindigkeitsverbesserungen
- **Parallelität**: Mehrere Anfragen gleichzeitig
- **Error Handling**: Robuste Fehlerbehandlung

### Wann Async verwenden:
- **Viele API-Anfragen**: Parallele Verarbeitung spart Zeit
- **I/O-intensive Operationen**: Während Netzwerk-Wartezeiten
- **Real-time Anwendungen**: Für bessere Responsivität

**Weiter mit Notebook 6: Datenvisualisierung! 📊**